In [2]:
import os
import xarray as xr

# === Paths ===
input_folder = r"SILO - Australian climate data\daily rain"  # Change for each variable
output_folder = r"SILO - Australian climate data\monthly rain"
os.makedirs(output_folder, exist_ok=True)

# === List .nc files ===
nc_files = [f for f in os.listdir(input_folder) if f.endswith(".nc")]

# === Process each file ===
for nc_file in nc_files:
    nc_path = os.path.join(input_folder, nc_file)
    print(f"\n📂 Processing: {nc_file}")

    try:
        # === Load dataset ===
        ds = xr.open_dataset(nc_path)
        
        # === Keep only numeric variables ===
        numeric_vars = [
            var for var in ds.data_vars 
            if ds[var].dtype in ["float32", "float64", "int32", "int64"]
        ]

        # === Monthly average ===
        ds_monthly = ds[numeric_vars].resample(time="MS").mean()

        # === Preserve coordinates lat and lon ===
        for coord in ["lat", "lon"]:
            if coord in ds:
                ds_monthly[coord] = ds[coord]

        # === Save file with _monthly suffix ===
        name, ext = os.path.splitext(nc_file)
        new_filename = f"{name}_monthly{ext}"
        output_path = os.path.join(output_folder, new_filename)

        ds_monthly.to_netcdf(output_path)
        print(f"✅ Saved monthly average file: {output_path}")

    except Exception as e:
        print(f"❌ Failed for {nc_file}: {e}")



📂 Processing: 1991.daily_rain.nc
✅ Saved monthly average file: SILO - Australian climate data\monthly rain\1991.daily_rain_monthly.nc
